In [1]:
import os
import pickle
from tqdm import tqdm
import mediapipe as mp
import cv2

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

DATA_DIR = './data/'
data = []
labels = []



# Loop through each data file
for data_file in os.listdir(DATA_DIR):
    data_path = os.path.join(DATA_DIR, data_file)
    if os.path.isdir(data_path):
        for subdirectory in os.listdir(data_path):
            category_dir = os.path.join(data_path, subdirectory)
            if os.path.isdir(category_dir):
                for img_path in os.listdir(category_dir):
                    data_aux = []
                    x_ = []
                    y_ = []

                    img = cv2.imread(os.path.join(category_dir, img_path).replace('\\', '/'))
                    if img is not None:
                        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        results = hands.process(img_rgb)

                        if results.multi_hand_landmarks:
                            for hand_landmarks in results.multi_hand_landmarks:
                                for landmark in hand_landmarks.landmark:
                                    x = landmark.x
                                    y = landmark.y
                                    x_.append(x)
                                    y_.append(y)

                            for landmark in hand_landmarks.landmark:
                                x = landmark.x
                                y = landmark.y
                                data_aux.append(x - min(x_))
                                data_aux.append(y - min(y_))

                            data.append(data_aux)
                            labels.append(subdirectory)

                            

                        else:
                            print(f"No hand landmarks detected in {data_file} => {subdirectory} =>{img_path}")
                    else:
                        print(f"Failed to load image: {img_path}")

# Save all the data and labels in a single pickle file
with open('data.pickle', 'wb') as f:
    pickle.dump({'data': data, 'labels': labels}, f)
